In [1]:
import pickle as pkl
data = pkl.load(open('favorited_dict.p', 'rb'))
max_item, max_user = pkl.load(open('useful_stats.p', 'rb'))
visual_data = pkl.load(open('id_feature_dict_davis.p', 'rb'))

In [2]:
from tqdm import tqdm_notebook as tqdm
import numpy as np

In [3]:
removed_items = set()
for key in range(max_item):
    if key not in visual_data:
        removed_items.add(key)

In [4]:
users_to_remove = []
for user in data:
    images = np.unique(data[user]).tolist()
    images = [image for image in images if image in visual_data]
    if not images: users_to_remove.append(user)
    else: data[user] = images

for user in users_to_remove:
    data.pop(user)

In [5]:
def create_validation(data):
    valid = {}
    for key in data:
        rated_items = data[key]
        if len(rated_items) > 1:
            valid[key] = np.random.choice(rated_items)
        else:
            valid[key] = None
    return valid
        

In [6]:
valid = create_validation(data)

In [7]:
def sample_triple(data, valid, nSamples=1):
    samples = np.zeros((nSamples, 3), dtype=int)
    keys = list(data.keys())
    users = np.random.choice(keys, nSamples) 
    samples[:, 0] = users
    
    for i, user in enumerate(tqdm(users)):
        rated_items = data[user]
        valid_item = valid[user]
        unrated_item = np.random.choice(max_item)
        while unrated_item in rated_items or unrated_item in removed_items or unrated_item == valid_item:
            unrated_item = np.random.choice(max_item)
            
        rated_item = np.random.choice(rated_items)
        while rated_item == valid_item:
            rated_item = np.random.choice(rated_items)

        samples[i, 1] = rated_item
        samples[i, 2] = unrated_item
            
    return samples


In [8]:
import math

def sigmoid(x):
    try:
        if type(x) in (int, float, np.float64):
            return 1 / (1 + math.exp(-x))
        else:
            if len(x) == 1:
                x = np.squeeze(x)
                return 1 / (1 + math.exp(-x))
            else:
                return 1 / (1 + np.exp(-x))
    except OverflowError:
        return 0

In [9]:
#hyperparameters
k=10
n=10
lr = 0.0001 #0.0001
lam_u = .01
lam_bias = .01
lam_rated = .01
lam_unrated = .01
lam_vf = .1
lam_E = .10
lam_vu = .10

sf = 1

In [10]:
def create_valid_dataset(data, valid, nItems=10): #nItems is the number of items not in train/ valid 
    
    valid_list = list(valid.items())
    valid_users = [x[0] for x in valid_list if x[1] is not None]
    valid_items = [x[1] for x in valid_list if x[1] is not None]
    valid_users = np.repeat(valid_users, nItems)
    valid_items = np.repeat(valid_items, nItems)
    
    samples = np.zeros((len(valid_users), 3), dtype=int)
    samples[:, 0] = valid_users
    samples[:, 1] = valid_items
    
    for i, user in enumerate(valid_users):
        rated_items = data[user]
        valid_item = valid[user]
        unrated_item = np.random.choice(max_item)
        while unrated_item in rated_items or unrated_item in removed_items or unrated_item == valid_item:
            unrated_item = np.random.choice(max_item)
            
        samples[i, 2] = unrated_item
        
    return samples



In [11]:
import tensorflow as tf

In [12]:
init = tf.truncated_normal_initializer(stddev = 0.1)

item_biases = tf.get_variable(name="item_biases", shape=max_item, initializer=init)
latent_items = tf.get_variable(name="item_latent_factors", shape=(max_item, k), initializer=init)
latent_users = tf.get_variable(name="user_latent_factors", shape=(max_user, k), initializer=init)
visual_users = tf.get_variable(name="user_visual_factors", shape=(max_user, n), initializer=init)
E = tf.get_variable(name="encoding_matrix", shape=(4096, n), initializer=init)
visual_bias = tf.get_variable(name="visual_bias", shape=(4096), initializer=init)

user = tf.placeholder(tf.int32, shape = [])
rated_item = tf.placeholder(tf.int32, shape = [])
unrated_item = tf.placeholder(tf.int32, shape = [])



In [29]:
init= tf.contrib.lookup.KeyValueTensorInitializer(keys=np.array([key for key in visual_data]), 
                                                  values = np.array([visual_data[key] for key in visual_data]))


hashtable = tf.contrib.lookup.HashTable(initializer=init, default_value=-1)

ValueError: Shapes must be equal rank, but are 1 and 2 for 'key_value_init_2' (op: 'InitializeTable') with input shapes: [2], [63988], [63988,4096].

In [27]:
values = np.array([visual_data[key] for key in visual_data])

In [ ]:
def wrapper(item_biases, latent_items, latent_users, visual_users, E, visual_bias,
            user, rated_item, unrated_item):

In [20]:
def BR_opt(item_biases, latent_items, latent_users, visual_users, E, visual_bias,
            user, rated_item, unrated_item):
    
    

    bias_difference = item_biases[rated_item] - item_biases[unrated_item]
    latent_rated_item = latent_items[rated_item]; latent_unrated_item = latent_items[unrated_item]
    latent_user = latent_users[user]
    
    latent_difference = tf.reduce_sum(tf.multiply(latent_rated_item - latent_unrated_item, latent_user))
        
    rated_vf = visual_data[rated_item]; unrated_vf = visual_data[unrated_item]
    vf_difference = rated_vf - unrated_vf
    
    vf_encoded = tf.matmul(E, np.reshape(vf_difference, (-1,1)), transpose_a=True)
    visual_difference = tf.reduce_sum(tf.multiply(vf_encoded, visual_users[user]))
    visual_bias_difference = tf.reduce_sum(tf.multiply(vf_difference, visual_bias))
    
    return (bias_difference + latent_difference + sf*visual_difference + sf*visual_bias_difference)

In [21]:
optimizer = tf.train.GradientDescentOptimizer(0.0007)

In [22]:
obj = BR_opt(item_biases, latent_items, latent_users, visual_users, E, visual_bias,
            user, rated_item, unrated_item)


ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [ ]:
def AUC(data, valid, item_biases, latent_items, latent_users, visual_users, E, visual_bias):

    samples = create_valid_dataset(data, valid, 5)

    auc = 0.0
    for user, rated_item, unrated_item in tqdm(samples):

        # Extract out the relevant variables
        rated_item_bias = item_biases[rated_item]
        unrated_item_bias = item_biases[unrated_item]
        latent_user = latent_users[user]
        latent_rated_item = latent_items[rated_item]
        latent_unrated_item = latent_items[unrated_item]

        visual_user = visual_users[user]
        rated_vf = visual_data[rated_item]
        unrated_vf = visual_data[unrated_item]
        vf_difference = rated_vf-unrated_vf
        # Determine the output, i.e. x_uij 
        br_opt = BR_opt(rated_item_bias, unrated_item_bias, latent_user, latent_rated_item, latent_unrated_item,
                        E, vf_difference, visual_user, visual_bias)

        auc += (br_opt > 0)
    return auc/len(samples)

In [ ]:
aucs = []

In [ ]:
samples = sample_triple(data, valid, 2500)
count = 0

for user, rated_item, unrated_item in tqdm(samples):

    # Extract out the relevant variables
    rated_item_bias = item_biases[rated_item]
    unrated_item_bias = item_biases[unrated_item]
    latent_user = latent_users[user]
    latent_rated_item = latent_items[rated_item]
    latent_unrated_item = latent_items[unrated_item]

    visual_user = visual_users[user]
    rated_vf = visual_data[rated_item]
    unrated_vf = visual_data[unrated_item]
    vf_difference = rated_vf-unrated_vf
    # Determine the output, i.e. x_uij 
    br_opt = BR_opt(rated_item_bias, unrated_item_bias, latent_user, latent_rated_item, latent_unrated_item,
                    E, vf_difference, visual_user, visual_bias)

    output = sigmoid(-br_opt)

    # Perform gradient updates
    
    item_biases[rated_item] = (1-lr*lam_bias) * rated_item_bias + lr * output
    item_biases[unrated_item] = (1-lr*lam_bias) * unrated_item_bias - lr * output

    latent_items[rated_item] = (1-lr*lam_rated) * latent_rated_item + lr * output * latent_user 
    latent_items[unrated_item] = (1-lr*lam_unrated) * latent_unrated_item - lr * output * latent_user 

    latent_users[user] = (1-lr*lam_unrated) * latent_user \
                        + lr * output * (latent_rated_item-latent_unrated_item) 

        
    E = (1-lr*lam_E) * E + sf*lr* output * (np.dot((vf_difference)[:,None],visual_user[None,:])) 

    visual_users[user] = (1-lr*lam_vu) * visual_user + sf*lr * output * np.dot(np.transpose(E), vf_difference) 
    
    visual_bias = (1-lr*lam_vf)*visual_bias + sf*lr * output * (vf_difference)



    count += 1
    if count % 250000 is 0: pass
        #aucs.append(AUC(data, valid, item_biases, latent_items, latent_users, visual_users, E, visual_bias))


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(np.median(np.array(vu_grad), axis=1))
plt.plot(np.median(np.array(lu_grad), axis=1))
plt.plot(ib_grad[0:100])
plt.plot(np.median(np.array(vb_grad), axis=1))
plt.plot(np.median(np.array(E_grad), axis=(1,2)))
plt.plot(np.median(np.array(li_grad), axis=1))

In [ ]:
np.max(E)

In [ ]:
plt.plot(ib_grad)
plt.plot(np.transpose(np.array(vb_grad))[0])

In [ ]:
plt.plot(np.array(range(len(aucs)+1))*250000, [0.5]+aucs)

In [ ]:
AUC(data, valid, item_biases, latent_items, latent_users, visual_users, E, visual_bias)

item_biases = np.random.normal(size=max_item)
latent_items = np.random.normal(size=(max_item, k))
latent_users = np.random.normal(size=(max_user, k))
visual_users = np.random.normal(size=(max_user, n))
E = np.random.normal(size=(4096, n))
visual_bias = np.random.normal(size=(4096))

AUC(data, valid, item_biases, latent_items, latent_users, visual_users, E, visual_bias)